In [1]:
import torch
import operator
import torch.nn as nn
from functools import reduce
from torch.autograd import Variable
from functools import partial
from collections import OrderedDict
import torch.nn.functional as F

import torch
import torch.nn as nn
from IPython.display import Image
import time
import torch.nn.functional as F
from functools import partial
from torch.autograd import Variable
from collections import OrderedDict
from torch.autograd import Variable

import os
# import cv2
import six
import time
import math
import torch
import random
import numbers
import warnings
import collections
import torchvision
import numpy as np
from torch import nn
from PIL import Image
# from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import ImageOps
import scipy.misc as misc
from torch.utils import data
from datetime import datetime
import torch.nn.functional as F
from __future__ import division
from torch.autograd import Variable
from torch.optim import lr_scheduler
from collections import OrderedDict
import torchvision.transforms as tfs
# from tensorboardX import SummaryWriter
from torchvision import datasets, models
import torchvision.transforms.functional as tf
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings("ignore")

from torchvision.transforms import ToTensor, ToPILImage
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad

# from functools import partial
# try:
#     import pydensecrf.densecrf as dcrf
# except:
#     print("Failed to import pydensecrf, CRF post-processing will not work")

count_ops = 0
count_params = 0

In [2]:
torch.__version__

'0.2.1+a4fc05a'

In [3]:
def get_num_gen(gen):
    return sum(1 for x in gen)


def is_pruned(layer):
    try:
        layer.mask
        return True
    except AttributeError:
        return False


def is_leaf(model):
    return get_num_gen(model.children()) == 0


def get_layer_info(layer):
    layer_str = str(layer)
    type_name = layer_str[:layer_str.find('(')].strip()
    return type_name


def get_layer_param(model):
    return sum([reduce(operator.mul, i.size(), 1) for i in model.parameters()])


### The input batch size should be 1 to call this function
def measure_layer(layer, x):
    global count_ops, count_params
    delta_ops = 0
    delta_params = 0
    multi_add = 1
    type_name = get_layer_info(layer)

    ### ops_conv
    if type_name in ['Conv2d','ConvTranspose2d']:
        out_h = int((x.size()[2] + 2 * layer.padding[0] - layer.kernel_size[0]) /
                    layer.stride[0] + 1)
        out_w = int((x.size()[3] + 2 * layer.padding[1] - layer.kernel_size[1]) /
                    layer.stride[1] + 1)
        delta_ops = layer.in_channels * layer.out_channels * layer.kernel_size[0] *  \
                layer.kernel_size[1] * out_h * out_w / layer.groups * multi_add
        delta_params = get_layer_param(layer)

    ### ops_nonlinearity
    elif type_name in ['ReLU', 'PReLU','ReLU6', 'LeakyReLU', 'Sigmoid']:
        delta_ops = x.numel()
        delta_params = get_layer_param(layer)

    ### ops_pooling
    elif type_name in ['AvgPool2d']:
        in_w = x.size()[2]
        kernel_ops = layer.kernel_size * layer.kernel_size
        out_w = int((in_w + 2 * layer.padding - layer.kernel_size) / layer.stride + 1)
        out_h = int((in_w + 2 * layer.padding - layer.kernel_size) / layer.stride + 1)
        delta_ops = x.size()[0] * x.size()[1] * out_w * out_h * kernel_ops
        delta_params = get_layer_param(layer)

    elif type_name in ['AdaptiveAvgPool2d']:
        delta_ops = x.size()[0] * x.size()[1] * x.size()[2] * x.size()[3]
        delta_params = get_layer_param(layer)

    ### ops_linear
    elif type_name in ['Linear']:
        weight_ops = layer.weight.numel() * multi_add
        bias_ops = layer.bias.numel()
        delta_ops = x.size()[0] * (weight_ops + bias_ops)
        delta_params = get_layer_param(layer)

    ### ops_nothing
    elif type_name in ['BatchNorm2d', 'Dropout2d', 'DropChannel',"ShuffleBlock",
                       'Dropout', 'InPlaceABN', 'InPlaceABNSync', 'Upsample', 'MaxPool2d']:
        delta_params = get_layer_param(layer)

    ### unknown layer type
    else:
        raise TypeError('unknown layer type: %s' % type_name)

    count_ops += delta_ops
    count_params += delta_params
    return

In [4]:
def measure_model(model, H, W):
    global count_ops, count_params
    count_ops = 0
    count_params = 0
    data = Variable(torch.zeros(1, 3, H, W).cuda())

    def should_measure(x):
        return is_leaf(x) or is_pruned(x)

    def modify_forward(model):
        for child in model.children():
            if should_measure(child):
                def new_forward(m):
                    def lambda_forward(x):
                        measure_layer(m, x)
                        return m.old_forward(x)
                    return lambda_forward
                child.old_forward = child.forward
                child.forward = new_forward(child)
            else:
                modify_forward(child)

    def restore_forward(model):
        for child in model.children():
            # leaf node
            if is_leaf(child) and hasattr(child, 'old_forward'):
                child.forward = child.old_forward
                child.old_forward = None
            else:
                restore_forward(child)

    modify_forward(model)
    model.forward(data)
    restore_forward(model)

    return count_ops, count_params

In [5]:
#加载预训练的mobilenetv2
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )

class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, dalited):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        self.d = dalited
        assert stride in [1, 2]

        self.use_res_connect = self.stride == 1 and inp == oup

        self.conv = nn.Sequential(
            # pw
            nn.Conv2d(inp, inp * expand_ratio, 1, 1, 0, bias=False),
            nn.BatchNorm2d(inp * expand_ratio),
            nn.ReLU6(inplace=True),
            # dw
            nn.Conv2d(inp * expand_ratio, inp * expand_ratio, 3, stride, padding=self.d, dilation=self.d, groups=inp * expand_ratio, bias=False),
            nn.BatchNorm2d(inp * expand_ratio),
            nn.ReLU6(inplace=True),
            # pw-linear
            nn.Conv2d(inp * expand_ratio, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileNetV2(nn.Module):
    def __init__(self, n_class=1000, input_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        # setting of inverted residual blocks
        self.interverted_residual_setting = [
            # t, c, n, s, d
            [1, 16, 1, 1, 1],    # 1/2
            [6, 24, 2, 2, 1],    # 1/4
            [6, 32, 3, 2, 1],    # 1/8
            [6, 64, 4, 1, 2],    # 1/8
            [6, 96, 3, 1, 3],    # 1/8
            [6, 160, 3, 1, 5],   # 1/8
            [6, 320, 1, 1, 7],  # 1/8
        ]

        # building first layer
        assert input_size % 32 == 0
        input_channel = int(32 * width_mult)
        self.last_channel = int(1280 * width_mult) if width_mult > 1.0 else 1280
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        for t, c, n, s ,d in self.interverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(InvertedResidual(input_channel, output_channel, s, t,d))
                else:
                    self.features.append(InvertedResidual(input_channel, output_channel, 1, t,d))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        self.features.append(nn.AvgPool2d(input_size/32))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.last_channel)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
                
                
net = MobileNetV2(n_class=1000)
mobilenet_v2 = list(net.features.children())
len(mobilenet_v2)

def channel_shuffle(x, groups):
    batchsize, num_channels, height, width = x.data.size()
    assert (num_channels % groups == 0)
    channels_per_group = num_channels // groups
    # reshape
    x = x.view(batchsize, groups, channels_per_group, height, width)
    # transpose
    x = torch.transpose(x, 1, 2).contiguous()
    # flatten
    x = x.view(batchsize, -1, height, width)
    return x

class ShuffleBlock(nn.Module):
    def __init__(self, groups):
        super(ShuffleBlock, self).__init__()
        self.groups = groups

    def forward(self, x):
        '''Channel shuffle: [N,C,H,W] -> [N,g,C/g,H,W] -> [N,C/g,g,H,w] -> [N,C,H,W]'''
        N,C,H,W = x.size()
        g = self.groups
        return x.view(N,g,int(C/g),H,W).permute(0,2,1,3,4).contiguous().view(N,C,H,W)


class ABN(nn.Sequential):
    def __init__(self, num_features):
        super(ABN, self).__init__(OrderedDict([
            ("bn",  nn.BatchNorm2d(num_features,eps=1e-05, momentum=0.1, affine=True)),
            ("act", nn.PReLU(num_features))
        ]))

class DSP(nn.Module):
    def __init__(self, inplanes, outplanes, c_tag=0.2, groups=4, dilation=(1,2,3,4)):
        super(DSP, self).__init__()
        
        self.out_c = round(c_tag * outplanes)
        
        self.down = nn.Sequential(
                nn.Conv2d(inplanes, self.out_c, 1, stride=1, groups=groups, bias=False),
                ABN(self.out_c)
          )
        
        self.pool =  nn.Sequential(
                                nn.AdaptiveAvgPool2d((1, 1)),
                                nn.Conv2d(self.out_c, self.out_c, 1, stride=1),
                                nn.BatchNorm2d(self.out_c,eps=1e-05, momentum=0.1, affine=True)
            )
        
        self.branch_1 = nn.Sequential(
                nn.Conv2d(self.out_c, self.out_c, kernel_size=3, padding=dilation[0], dilation = dilation[0],groups=self.out_c, bias=False),
                ABN(self.out_c),
                nn.Conv2d(self.out_c, self.out_c, kernel_size=1, bias=False),
                nn.BatchNorm2d(self.out_c,eps=1e-05, momentum=0.1, affine=True)
        )
        self.branch_2 = nn.Sequential(
                nn.Conv2d(self.out_c, self.out_c, kernel_size=3, padding=dilation[1],dilation = dilation[1],groups=self.out_c, bias=False),
                ABN(self.out_c),
                nn.Conv2d(self.out_c, self.out_c, kernel_size=1, bias=False),
                nn.BatchNorm2d(self.out_c,eps=1e-05, momentum=0.1, affine=True)
            )
        self.branch_3 = nn.Sequential(
                nn.Conv2d(self.out_c, self.out_c, kernel_size=3, padding=dilation[2],dilation = dilation[2],groups=self.out_c, bias=False),
                ABN(self.out_c),
                nn.Conv2d(self.out_c, self.out_c, kernel_size=1, bias=False),
                nn.BatchNorm2d(self.out_c,eps=1e-05, momentum=0.1, affine=True)
        )
        
        self.branch_4 = nn.Sequential(
                nn.Conv2d(self.out_c, self.out_c, kernel_size=3, padding=dilation[3],dilation = dilation[3],groups=self.out_c, bias=False),
                ABN(self.out_c),
                nn.Conv2d(self.out_c, self.out_c, kernel_size=1, bias=False),
                nn.BatchNorm2d(self.out_c,eps=1e-05, momentum=0.1, affine=True)
        )
       
        self.groups = groups
        
        self.module_act = nn.PReLU(outplanes)
      
    def forward(self, x):
        input_x = x
        x_size = x.size()
        x= self.down(x)
        branch_1 = self.branch_1(x)
        branch_2 = self.branch_2(x)
        branch_3 = self.branch_3(x)
        branch_4 = self.branch_4(x)
        pool = F.upsample(self.pool(x),x_size[2:], mode= "bilinear")
        out = channel_shuffle(torch.cat((branch_1,branch_2,branch_3,branch_4,pool), 1), self.groups)
        
        if out.size() == input_x.size():
            out = out + input_x
        return self.module_act(out)
    
class MCIM(nn.Module):
    def __init__(self, in_chs = 320, out_chs = 320):
        super(MCIM, self).__init__()
        
        self.pool =  nn.Sequential(
                                nn.AdaptiveAvgPool2d((1, 1)),
                                nn.Conv2d(in_chs, 80, 1, stride=1),
                                nn.BatchNorm2d(80),
                                nn.LeakyReLU(0.1)
            )
        
        self.conv_small = DSP(in_chs, out_chs, dilation = (1,2,3,5))
        
        self.conv_middle = DSP(out_chs, out_chs, dilation = (7,9,11,13))

        self.conv_larger = DSP(out_chs, out_chs, dilation = (17,19,21,23))
   
    def forward(self, x):
        x_size = x.size()
        small = self.conv_small(x)
        middle = self.conv_middle(small)
        larger = self.conv_larger(middle)
        pool = F.upsample(self.pool(x),x_size[2:], mode= "bilinear")
        output = small+middle+larger
        output = torch.cat([output,pool],1)
        return output
    
    
#语义分割模型 mobilenet_v2
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()

class Attention_Block(nn.Module):
    def __init__(self, channel, reduction=16):
        super(Attention_Block, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.channel_excitation = nn.Sequential(nn.Linear(channel, int(channel//reduction)),
                                                nn.ReLU(inplace=True),                                             
                                                nn.Linear(int(channel//reduction), channel),
                                                nn.Sigmoid())
    def forward(self, x):
        bahs, chs, _, _ = x.size()
        chn_se = self.avg_pool(x).view(bahs, chs)        
        return self.channel_excitation(chn_se).view(bahs, chs, 1, 1)

                
class M2_semantic(nn.Module):
    def __init__(self, num_classes=19):
        super(M2_semantic, self).__init__()
        
        # building inverted residual blocks
        self.mod1 = mobilenet_v2[0]
        self.mod2 = mobilenet_v2[1]
        self.mod3 = nn.Sequential(mobilenet_v2[2],mobilenet_v2[3])
        self.mod4 = nn.Sequential(mobilenet_v2[4],mobilenet_v2[5],mobilenet_v2[6])
        self.mod5 = nn.Sequential(mobilenet_v2[7],mobilenet_v2[8],mobilenet_v2[9],mobilenet_v2[10])
        self.mod6 = nn.Sequential(mobilenet_v2[11],mobilenet_v2[12],mobilenet_v2[13])
        self.mod7 = nn.Sequential(mobilenet_v2[14],mobilenet_v2[15],mobilenet_v2[16])
        self.mod8 = mobilenet_v2[17]
        
        self.LRM =  nn.Sequential(
                        nn.Conv2d(32, 32, kernel_size=3, padding = 1, groups = 32), 
                        ABN(32),
                        nn.Conv2d(32, 160, kernel_size=1),    
                        ABN(160)
                            )
        
        self.multi_scale = MCIM(320, 320)
        
        self.attention = Attention_Block(160)
        
        self.final = nn.Sequential(
            nn.Conv2d(400, 256, kernel_size = 1, stride = 1,groups =16),
            ShuffleBlock(16),
            nn.BatchNorm2d(256,eps=1e-05, momentum=0.1, affine=True),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),
            nn.Conv2d(256, num_classes, kernel_size=1)
        )
        initialize_weights(self.LRM, self.attention, self.multi_scale, self.final)

    def forward(self, x):
        x_size = x.size()  
        stg0 = self.mod1(x)         # torch.Size([1, 32, 112, 224])
        
        stg1 = self.mod2(stg0)      # torch.Size([1, 16, 112, 224])
        stg2 = self.mod3(stg1)      # torch.Size([1, 24, 56, 112])     
        stg3 = self.mod4(stg2)  # torch.Size([1, 32, 28, 56]) 
        
        LRM = self.LRM(stg3)  # torch.Size([1, 32, 28, 56])
        
        stg4 = self.mod5(stg3)  # torch.Size([1, 64, 28, 56])
        stg5 = self.mod6(stg4) # torch.Size([1, 96, 28, 56])   
        stg6 = self.mod7(stg5)  # torch.Size([1, 160, 28, 56])
        
        attention = self.attention(stg6)
        
        stg6 = torch.mul(attention,LRM) + stg6
        
        stg7 = self.mod8(stg6)  # torch.Size([1, 320, 28, 56])

        multi_scale = self.multi_scale(stg7)
        
#         print(modified_aspp.size())
        out = self.final(multi_scale)
        
        return F.upsample(out, x_size[2:], mode='bilinear')
    
model = M2_semantic(num_classes=21).cuda()

In [6]:
net_h, net_w = 360,640
count_ops, count_params = measure_model(model, net_h, net_w)
print('FLOPs: {}, Params: {}'.format(count_ops, count_params))

FLOPs: 7267744548.0, Params: 1950415


In [7]:
net_h, net_w = 512,512
count_ops, count_params = measure_model(model, net_h, net_w)
print('FLOPs: {}, Params: {}'.format(count_ops, count_params))

FLOPs: 8260974628.0, Params: 1950415


In [6]:
net_h, net_w = 713,713
count_ops, count_params = measure_model(model, net_h, net_w)
print('FLOPs: {}, Params: {}'.format(count_ops, count_params))

FLOPs: 16277781732.0, Params: 1950415


In [6]:
net_h, net_w = 1024,448
count_ops, count_params = measure_model(model, net_h, net_w)
print('FLOPs: {}, Params: {}'.format(count_ops, count_params))

FLOPs: 14431301668.0, Params: 1950415


In [6]:
net_h, net_w = 1024,512
count_ops, count_params = measure_model(model, net_h, net_w)
print('FLOPs: {}, Params: {}'.format(count_ops, count_params))

FLOPs: 16482744356.0, Params: 1950415
